## Imports

In [1]:
import copy
import logging
import os
from pathlib import Path
from typing import Any, Dict, List, Optional

from model_merging.data.dataset import HFImageClassification
from model_merging.model.image_classifier import ImageClassifier
import open_clip
import wandb

import hydra
import omegaconf
import pytorch_lightning as pl
import torch
from hydra import compose, initialize
from hydra.utils import instantiate
from lightning.pytorch import Callback
from omegaconf import DictConfig, ListConfig, OmegaConf
from torch.nn.utils import parameters_to_vector, vector_to_parameters

from nn_core.callbacks import NNTemplateCore
from nn_core.common import PROJECT_ROOT
from nn_core.common.utils import enforce_tags, seed_index_everything
from nn_core.model_logging import NNLogger
from nn_core.serialization import NNCheckpointIO

# Force the execution of __init__.py if this file is executed directly.
import model_merging  # noqa
from model_merging.model.encoder import ClassificationHead, ImageEncoder
from model_merging.model.heads import (
    get_classification_head,
)
from model_merging.utils.io_utils import (
    boilerplate,
    load_model_from_hf,
)
from model_merging.utils.plots import plot_interactive_radar_chart
from model_merging.utils.utils import (
    build_callbacks,
    get_finetuning_accuracies,
    compute_avg_accuracy,
    print_memory,
)
import json
import os

/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[12:05:15] INFO     PyTorch version 2.8.0 available.                                                   ]8;id=593326;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/datasets/config.py\config.py]8;;\:]8;id=372979;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/datasets/config.py#54\54]8;;\

/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/imports.py:14: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/data/datamodule.py:206: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "con

In [2]:
import hydra
from hydra import initialize, compose
from typing import Dict, List

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path=str("../conf"), job_name="layer_analysis")
cfg = compose(config_name="multitask", overrides=["benchmark=hard"])

/media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/IPython/core/async_helpers.py:129: UserWarning: 
'hydra/launcher/basic' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/automatic_schema_matching for migration instructions.
  coro.send(None)


## Boilerplate

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
pylogger = logging.getLogger(__name__)

In [5]:
seed_index_everything(cfg)

logger, template_core = boilerplate(cfg)

num_tasks = len(cfg.benchmark.datasets)

# Temporarily disable struct mode to allow dynamic update
omegaconf.OmegaConf.set_struct(cfg, False)
cfg.num_tasks = num_tasks  # Now we can safely update it
omegaconf.OmegaConf.set_struct(cfg, True)  # Re-enable struct mode

# upperbound accuracies, used for logging the normalized accuracy
finetuned_accuracies: Dict[str, float] = get_finetuning_accuracies(
    cfg.misc.finetuned_accuracy_path
)[cfg.nn.encoder.model_name]

[12:05:25] INFO     Global seed set to 1608637542                                                        ]8;id=100288;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_fabric/utilities/seed.py\seed.py]8;;\:]8;id=578695;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_fabric/utilities/seed.py#53\53]8;;\

           INFO     Setting seed 1608637542 from seeds[0]                                              ]8;id=506654;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/common/utils.py\utils.py]8;;\:]8;id=125355;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/common/utils.py#107\107]8;;\

           INFO     Tags: ['dev']                                                                       ]8;id=507214;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/common/utils.py\utils.py]8;;\:]8;id=552910;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/common/utils.py#96\96]8;;\

           INFO     Instantiating <WandbLogger>                                                 ]8;id=346224;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/model_logging.py\model_logging.py]8;;\:]8;id=130475;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/model_logging.py#41\41]8;;\

           ERROR    Failed to detect the name of this notebook, you can set it manually with the     ]8;id=170108;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/wandb/jupyter.py\jupyter.py]8;;\:]8;id=507338;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/wandb/jupyter.py#224\224]8;;\
                    WANDB_NOTEBOOK_NAME environment variable to enable code saving.                                

wandb: Currently logged in as: crisostomi (gladia). Use `wandb login --relogin` to force relogin


[12:05:27] INFO     Uploading source code to W&B                                                ]8;id=635995;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/model_logging.py\model_logging.py]8;;\:]8;id=25814;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/nn_core/model_logging.py#63\63]8;;\

## Load models

In [6]:
# only has vision encoder, no text transformer
base_model: ImageEncoder = load_model_from_hf(
    model_name=cfg.nn.encoder.model_name
)

finetuned_models = {
    dataset: load_model_from_hf(
        model_name=cfg.nn.encoder.model_name, dataset_name=dataset.name
    ).state_dict()
    for dataset in cfg.benchmark.datasets
}

pylogger.info(f"Number of tasks: {cfg.num_tasks}")
pylogger.info(f"Finetuned models: {list(finetuned_models.keys())}")


[12:05:32] INFO     Loading ViT-B-32 pre-trained weights.                                             ]8;id=591644;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=663784;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#21\21]8;;\

           INFO     Loading pretrained ViT-B-32 from OpenAI.                                          ]8;id=560796;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py\factory.py]8;;\:]8;id=355368;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

[12:05:33] INFO     Removing text transformer from the model.                                         ]8;id=527597;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=234146;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#35\35]8;;\

[12:05:34] INFO     Loading ViT-B-32 pre-trained weights.                                             ]8;id=836233;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=748715;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#21\21]8;;\

           INFO     Loading pretrained ViT-B-32 from OpenAI.                                          ]8;id=650903;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py\factory.py]8;;\:]8;id=922805;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

...

[12:05:42] INFO     Loading ViT-B-32 pre-trained weights.                                             ]8;id=434389;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=806390;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#21\21]8;;\

           INFO     Loading pretrained ViT-B-32 from OpenAI.                                          ]8;id=229256;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py\factory.py]8;;\:]8;id=261810;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

[12:05:43] INFO     Removing text transformer from the model.                                         ]8;id=673489;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py\encoder.py]8;;\:]8;id=756732;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/model/encoder.py#35\35]8;;\

           INFO     Number of tasks: 5                                                             ]8;id=198602;file:///tmp/ipykernel_4104257/2486514914.py\2486514914.py]8;;\:]8;id=853599;file:///tmp/ipykernel_4104257/2486514914.py#13\13]8;;\

           INFO     Finetuned models: [{'_target_': 'model_merging.data.dataset.load_dataset',     ]8;id=189023;file:///tmp/ipykernel_4104257/2486514914.py\2486514914.py]8;;\:]8;id=23087;file:///tmp/ipykernel_4104257/2486514914.py#14\14]8;;\
                    'name': 'Cars', 'preprocess_fn': '???', 'ft_epochs': 35, 'hf_dataset':                         
                    {'_target_': 'datasets.load_dataset', 'path': 'tanganke/stanford_cars'}},                      
                    {'_target_': 'model_merging.data.dataset.load_dataset', 'name': 'KMNIST',                      
                    'ft_epochs': 5, 'preprocess_fn': '???', 'hf_dataset': {'_target_':                             
                    'datasets.load_dataset', 'path': 'tanganke/kmnist'}}, {'_target_':                             
                    'model_merging.data.dataset.load_dataset', 'name': 'RESISC45', 'ft_epochs':                    
                    15, 'preprocess_fn': '???', 'hf_dataset': {'_target_':                                         
                    'datasets.load_dataset', 'path': 'tanganke/resisc45'}, 'label_map': {0: 0, 1:                  
                    2, 2: 3, 3: 5, 4: 6, 5: 7, 6: 9, 7: 10, 8: 11, 9: 12, 10: 13, 11: 14, 12: 15,                  
                    13: 1, 14: 16, 15: 17, 16: 18, 17: 19, 18: 4, 19: 20, 20: 21, 21: 22, 22: 23,                  
                    23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 8,                 
                    33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42:                    
                    42, 43: 43, 44: 44}, 'classnames_override': ['airplane', 'forest', 'airport',                  
                    'baseball diamond', 'industrial area', 'basketball court', 'beach', 'bridge',                  
                    'river', 'chaparral', 'church', 'circular farmland', 'cloud', 'commercial                      
                    area', 'dense residential', 'desert', 'freeway', 'golf course', 'ground track                  
                    field', 'harbor', 'intersection', 'island', 'lake', 'meadow', 'medium                          
                    residential', 'mobile home park', 'mountain', 'overpass', 'palace', 'parking                   
                    lot', 'railway', 'railway station', 'rectangular farmland', 'roundabout',                      
                    'runway', 'sea ice', 'ship', 'snowberg', 'sparse residential', 'stadium',                      
                    'storage tank', 'tennis court', 'terrace', 'thermal power station',                            
                    'wetland']}, {'_target_': 'model_merging.data.dataset.load_dataset', 'name':                   
                    'EMNIST', 'ft_epochs': 2, 'preprocess_fn': '???', 'hf_dataset': {'_target_':                   
                    'datasets.load_dataset', 'path': 'tanganke/emnist_mnist'}}, {'_target_':                       
                    'model_merging.data.dataset.load_dataset', 'name': 'MNIST', 'ft_epochs': 5,                    
                    'preprocess_fn': '??', 'hf_dataset': {'_target_': 'datasets.load_dataset',                     
                    'path': 'mnist'}}]                                                                             

## Merging

In [7]:
non_matrix_params_aggregation = 'mean'
compression_ratio = 1 / len(cfg.benchmark.datasets)

In [8]:
from model_merging.merging.structured import aggregate_decomposed_task_vectors, decompose_task_vectors, get_svd_dict
from model_merging.utils.utils import apply_dict_to_model, compute_task_dict

task_dicts = {}

for dataset in cfg.benchmark.datasets:
    task_dicts[dataset] = compute_task_dict(
        base_model.state_dict(), finetuned_models[dataset]
    )
    del finetuned_models[dataset]  # Delete one model at a time
    torch.cuda.empty_cache()

print_memory("after computing task dicts")

[12:05:44] WARNING  after computing task dicts -- memory in MB: 4318.53125                              ]8;id=293850;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/utils/utils.py\utils.py]8;;\:]8;id=161008;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/src/model_merging/utils/utils.py#23\23]8;;\

In [9]:
decomposed_task_vectors = decompose_task_vectors(task_dicts, compression_ratio)

Computing and compressing SVD: 100%|██████████| 5/5 [00:27<00:00,  5.58s/it]


In [ ]:
def aggregate_decomposed_task_vectors(ref_state_dict, decomposed_task_vectors, non_matrix_params_aggregation):
    # TODO: implement your own function here 
    pass 

In [ ]:

multi_task_vector = aggregate_decomposed_task_vectors(
    ref_state_dict=copy.deepcopy(base_model.state_dict()),
    decomposed_task_vectors=decomposed_task_vectors,
    non_matrix_params_aggregation=non_matrix_params_aggregation,
)


Aggregate (delta) w/ rotation propagation: 100%|██████████| 158/158 [00:03<00:00, 43.35it/s]


In [20]:
merged_encoder: ImageEncoder = copy.deepcopy(base_model)

merged_encoder = apply_dict_to_model(
    multi_task_vector,
    merged_encoder,
)

In [21]:
results = {}

for dataset_cfg in cfg.benchmark.datasets:

    dataset = instantiate(
        dataset_cfg, preprocess_fn=base_model.val_preprocess
    )

    classification_head = get_classification_head(
        cfg.nn.encoder.model_name,
        dataset_cfg.name,
        ckpt_path=cfg.misc.ckpt_path,
        openclip_cachedir=cfg.misc.openclip_cachedir,
        device=cfg.device,
    )

    model = ImageClassifier(
        encoder=merged_encoder,
        classifier=classification_head,
        x_key=cfg.conventions.x_key,
        y_key=cfg.conventions.y_key,
    )

    model.set_metrics(len(dataset.classnames))
    model.set_task(dataset_cfg.name)
    model.set_finetuning_accuracy(
        finetuned_accuracies[
            dataset_cfg.name + "Val" if cfg.eval_on_train else dataset_cfg.name
        ]
    )

    callbacks: List[Callback] = build_callbacks(cfg.train.callbacks, template_core)

    trainer = pl.Trainer(
        default_root_dir=cfg.core.storage_dir,
        plugins=[NNCheckpointIO(jailing_dir=logger.run_dir)],
        logger=logger,
        callbacks=callbacks,
        limit_test_batches=(
            cfg.number_of_train_batches if cfg.eval_on_train else None
        ),
        **cfg.train.trainer,
    )

    if cfg.eval_on_train:
        pylogger.error("For now evaluation supported only on val-set")
        pylogger.info(f"Evaluating on {dataset_cfg.name} the training set")
        test_results = trainer.test(model=model, dataloaders=dataset.train_loader)

    else:
        pylogger.info(f"Evaluating on the {dataset_cfg.name} test set!")
        test_results = trainer.test(model=model, dataloaders=dataset.test_loader)

    results[dataset_cfg.name] = test_results

avg = compute_avg_accuracy(results)
results["avg"] = [
    avg
]  # as a list for consistency due to lightning logging stuff this way

logger.experiment.log(avg)

pylogger.info(results)

INFO: GPU available: True (cuda), used: True


Loading classification head from ./models//ViT-B-32/head_Cars.pt


[12:18:07] INFO     GPU available: True (cuda), used: True                                          ]8;id=366245;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py\rank_zero.py]8;;\:]8;id=211517;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\

INFO: TPU available: False, using: 0 TPU cores


           INFO     TPU available: False, using: 0 TPU cores                                        ]8;id=254030;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py\rank_zero.py]8;;\:]8;id=910449;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\

...

[12:19:05] INFO     You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores.  ]8;id=961103;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py\rank_zero.py]8;;\:]8;id=280922;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\
                    To properly utilize them, you should set                                                       
                    `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off                   
                    precision for performance. For more details, read                                              
                    https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.ht                
                    ml#torch.set_float32_matmul_precision                                                          

           INFO     LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]                                            ]8;id=693228;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/pytorch_lightning/accelerators/cuda.py\cuda.py]8;;\:]8;id=412711;file:///media/donato/Extra-storage/Code/model-merging/libraries/model-merging/.venv/lib/python3.11/site-packages/pytorch_lightning/accelerators/cuda.py#57\57]8;;\

Testing DataLoader 0: 100%|██████████| 79/79 [00:08<00:00,  9.69it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      acc/test/MNIST       │    0.9639999866485596     │
│      loss/test/MNIST      │    0.14167176187038422    │
│ normalized_acc/test/MNIST │    0.9693313241004944     │
└───────────────────────────┴───────────────────────────┘

[12:19:16] INFO     {'Cars': [{'acc/test/Cars': 0.017410770058631897, 'loss/test/Cars':            ]8;id=924905;file:///tmp/ipykernel_4104257/1006171372.py\1006171372.py]8;;\:]8;id=39203;file:///tmp/ipykernel_4104257/1006171372.py#63\63]8;;\
                    6.439502716064453, 'normalized_acc/test/Cars': 0.02181364968419075}],                          
                    'KMNIST': [{'acc/test/KMNIST': 0.3395000100135803, 'loss/test/KMNIST':                         
                    1.981614112854004, 'normalized_acc/test/KMNIST': 0.34596964716911316}],                        
                    'RESISC45': [{'acc/test/RESISC45': 0.04523809626698494, 'loss/test/RESISC45':                  
                    5.1346235275268555, 'normalized_acc/test/RESISC45': 0.04746835678815842}],                     
                    'EMNIST': [{'acc/test/EMNIST': 0.9717000126838684, 'loss/test/EMNIST':                         
                    0.133066788315773, 'normalized_acc/test/EMNIST': 0.9770739078521729}],                         
                    'MNIST': [{'acc/test/MNIST': 0.9639999866485596, 'loss/test/MNIST':                            
                    0.14167176187038422, 'normalized_acc/test/MNIST': 0.9693313241004944}], 'avg':                 
                    [{'acc/test/avg': 0.467569775134325, 'normalized_acc/test/avg':                                
                    0.4723313771188259}]}                                                                          